In [2]:
import pandas as pd
import numpy as np

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.externals import joblib

# fix random seed for reproducibility
np.random.seed(7)

from keras.layers import Input, Dense, Embedding, merge, Convolution2D, MaxPooling2D, Dropout
from sklearn.cross_validation import train_test_split
from keras.layers.core import Reshape, Flatten
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Model

import xgboost as xgb
from sklearn.model_selection import train_test_split

Using TensorFlow backend.
/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Acquiring data

In [3]:
df = pd.read_csv('X_train.csv', encoding='utf-8')

## Splitting data

In [4]:
train, test = train_test_split(df)

## tfidf

In [5]:
tfidf = TfidfTransformer()

In [6]:
cv = CountVectorizer(max_features=100)
ft = tfidf.fit_transform(cv.fit_transform(train.comment))

In [7]:
t_ft = tfidf.fit_transform(cv.fit_transform(test.comment))

In [8]:
X_train = ft.toarray()
X_test = t_ft.toarray()
y_train = train.reting-1
y_test = test.reting-1

In [9]:
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)

In [14]:
%%time
param = {'silent':1, 'objective':'multi:softmax' }
param['nthread'] = 8
param['eval_metric'] = 'merror'
param['eta'] = 0.075
param['max_depth'] = 5
param['silent'] = 1
param['num_class'] = 5
param['n_estimators'] = 300
param['early_stopping_rounds'] = 30
param['learning_rates'] = 0.2

watchlist = [ (dtrain,'train'), (dtest, 'test') ]

bst = xgb.train(param, dtrain, 500, watchlist,verbose_eval=25)

[0]	train-merror:0.388024	test-merror:0.420067
[25]	train-merror:0.382635	test-merror:0.411855
[50]	train-merror:0.374337	test-merror:0.411085
[75]	train-merror:0.360907	test-merror:0.412368
[100]	train-merror:0.348075	test-merror:0.413395
[125]	train-merror:0.335415	test-merror:0.413908
[150]	train-merror:0.323268	test-merror:0.416987
[175]	train-merror:0.3142	test-merror:0.417501
[200]	train-merror:0.30633	test-merror:0.418014
[225]	train-merror:0.29923	test-merror:0.419553
[250]	train-merror:0.291531	test-merror:0.42212
[275]	train-merror:0.283319	test-merror:0.42135
[300]	train-merror:0.274337	test-merror:0.423146
[325]	train-merror:0.266553	test-merror:0.422889
[350]	train-merror:0.260137	test-merror:0.424172
[375]	train-merror:0.253807	test-merror:0.424172
[400]	train-merror:0.247648	test-merror:0.425199
[425]	train-merror:0.241061	test-merror:0.425455
[450]	train-merror:0.235928	test-merror:0.425199
[475]	train-merror:0.230111	test-merror:0.426739
CPU times: user 2min 43s, sys: 